In [10]:
from src import visualization as vis
from src import pre_process
from src import cnn

import os
import cv2
import matplotlib.pyplot as plt
from IPython.display import display
import warnings
warnings.filterwarnings("ignore")
#alejandro.gutierrez@sdggroup.com

In [11]:
def napari():
    vis.visualize_images("D:/TFM/Microscopy/video/3288-1.czi")
    vis.napari_view_splited_channels()

# napari()

In [12]:
# 1. Creation of dataframes for each channel independently and all together
blue_df, green_df, red_df, df = pre_process.create_df("D:/TFM/Microscopy/video/copies/channels/")
red = red_df.drop(columns = ["Sample", "Duplicates"])

def save_df(df_list, path_list, index = True):
    for df, path in zip(df_list, path_list): 
        df.to_csv(path, index = index)
        print(f'Dataframe has been saved to {path}')

df_list = [blue_df, green_df, red_df, df]
path_list = ["C:/Users/saraa/TFM/Microscopy/data/blue","C:/Users/saraa/TFM/Microscopy/data/green", "C:/Users/saraa/TFM/Microscopy/data/red","C:/Users/saraa/TFM/Microscopy/data/rgb"]
save_df(df_list, path_list)

['D:/TFM/Microscopy/video/copies/channels/blue', 'D:/TFM/Microscopy/video/copies/channels/green', 'D:/TFM/Microscopy/video/copies/channels/red']
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/blue
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/green
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/red
Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/rgb


In [14]:
def masks_df(input_stack, type, output_df):
    # 2. Generate masks for each microscopy image stack 
    cnn.mask(input_stack, type)
    # 3. Generate mask dataframe
    maskdf = pre_process.mask_df(output_df)
    return maskdf

input_stack = "D:\\TFM\\Microscopy\\video\\3288-1-AP-OP.czi"
type = "czi"
output_df = 'C:/Users/saraa/TFM/mask/nuclei/'
nucleiM = masks_df(input_stack, type, output_df)

(1, 201, 3, 1, 940, 940, 1)


100%|██████████| 603/603 [00:46<00:00, 12.89it/s]


In [17]:
df_list = [nucleiM]
path_list = ["C:/Users/saraa/TFM/Microscopy/data/nucleiM"]
save_df(df_list, path_list)

Dataframe has been saved to C:/Users/saraa/TFM/Microscopy/data/nucleiM


In [18]:
Xtrain, Xtest, Ytrain, Ytest = pre_process.train_valid_split(red, nucleiM)
pre_process.plot_training_data(Xtrain, Ytrain)

In [22]:
def split_shutil_train_val():
    # 1. Creating folders 
    data_path = "D:/TFM/Microscopy/video/copies/img/"
    folder_names= ["train_folder", "val_folder"]
    pre_process.createfolders(data_path,folder_names)

    data_path = "D:/TFM/Microscopy/video/copies/masks/"
    folder_names= ["train_folder", "val_folder"]
    pre_process.createfolders(data_path,folder_names)
    
    # 2. Splitting images data
    import shutil
    dfs = [Xtrain, Xtest]
    paths = ["D:/TFM/Microscopy/video/copies/img/train_folder/", "D:/TFM/Microscopy/video/copies/img/val_folder/"]
    for path, df in zip(paths, dfs):
        try: 
            pre_process.move_images(df, path)
        except shutil.Error:
            print(f"The files already exist in the destination path: {path}")
    
    # 3. Splitting masks data
    dfs = [Ytrain, Ytest]
    paths = ["D:/TFM/Microscopy/video/copies/masks/train_folder/", "D:/TFM/Microscopy/video/copies/masks/val_folder/"]
    for path, df in zip(paths, dfs):
        try: 
            pre_process.move_images(df, path)
        except shutil.Error:
            print(f"The files already exist in the destination path: {path}")  

# split_shutil_train_val()

In [23]:
def create_shutil_labels():
    # 1. Creating folders 
    train_path = "D:/TFM/Microscopy/video/copies/img/train_folder"
    val_path = "D:/TFM/Microscopy/video/copies/img/val_folder"
    # folder_names= ["red", "green", "blue"]
    folder_names= ["red"]
    pre_process.createfolders(train_path,folder_names)
    pre_process.createfolders(val_path,folder_names)

    train_path = "D:/TFM/Microscopy/video/copies/masks/train_folder"
    val_path = "D:/TFM/Microscopy/video/copies/masks/val_folder"
    # folder_names= ["red", "green", "blue"]
    folder_names= ["red"]
    pre_process.createfolders(train_path,folder_names)
    pre_process.createfolders(val_path,folder_names)
    
    # 2. Creating labels for training and testing data
    Xtrain["new_file_path"] = "D:/TFM/Microscopy/video/copies/img/train_folder/" + Xtrain["Image_id"] + ".tif"
    Xtrain["new_file_path"] = Xtrain["new_file_path"].astype(str)
    Xtest["new_file_path"] = "D:/TFM/Microscopy/video/copies/img/val_folder/" + Xtest["Image_id"] + ".tif"
    Xtest["new_file_path"] = Xtest["new_file_path"].astype(str)
    
    Ytrain["new_file_path"] = "D:/TFM/Microscopy/video/copies/masks/train_folder/" + Ytrain["Timepoint"] + ".png"
    Ytrain["new_file_path"] = Ytrain["new_file_path"].astype(str)
    Ytest["new_file_path"] = "D:/TFM/Microscopy/video/copies/masks/val_folder/" + Ytest["Timepoint"] + ".png"
    Ytest["new_file_path"] = Ytest["new_file_path"].astype(str)
    
    # 3. Save dataframe
    Xtrain.to_csv("C:/Users/saraa/TFM/Microscopy/data/img_training", index = True)
    Xtest.to_csv("C:/Users/saraa/TFM/Microscopy/data/img_testing", index = True)
    Ytrain.to_csv("C:/Users/saraa/TFM/Microscopy/data/mask_training", index = True)
    Ytest.to_csv("C:/Users/saraa/TFM/Microscopy/data/mask_testing", index = True)

    # 4. Splitting data for images 
    img_train_path_red = "D:/TFM/Microscopy/video/copies/img/train_folder/red"
    # train_path_green = "D:/TFM/Microscopy/video/copies/train_folder/green"
    # train_path_blue = "D:/TFM/Microscopy/video/copies/train_folder/blue"
    pre_process.images_class(Xtrain, img_train_path_red)    

    img_val_path_red = "D:/TFM/Microscopy/video/copies/img/val_folder/red"
    # val_path_green = "D:/TFM/Microscopy/video/copies/val_folder/green"
    # val_path_blue = "D:/TFM/Microscopy/video/copies/val_folder/blue"
    pre_process.images_class(Xtest, img_val_path_red)

    # 5. Splitting data for masks
    mask_train_path_red = "D:/TFM/Microscopy/video/copies/masks/train_folder/red"
    # train_path_green = "D:/TFM/Microscopy/video/copies/train_folder/green"
    # train_path_blue = "D:/TFM/Microscopy/video/copies/train_folder/blue"
    pre_process.images_class(Ytrain, mask_train_path_red)    

    mask_val_path_red = "D:/TFM/Microscopy/video/copies/masks/val_folder/red"
    # val_path_green = "D:/TFM/Microscopy/video/copies/val_folder/green"
    # val_path_blue = "D:/TFM/Microscopy/video/copies/val_folder/blue"
    pre_process.images_class(Ytest, mask_val_path_red)

# create_shutil_labels()    

In [24]:
img_train_path = "D:/TFM/Microscopy/video/copies/img/train_folder/"
img_val_path = "D:/TFM/Microscopy/video/copies/img/val_folder/"
mask_train_path = "D:/TFM/Microscopy/video/copies/masks/train_folder/"
mask_val_path = "D:/TFM/Microscopy/video/copies/masks/val_folder/"

img_gen, valid_img_gen, mask_gen, valid_mask_gen = pre_process.get_generator(img_train_path, img_val_path, 
                                                                             mask_train_path, mask_val_path)

Found 160 images belonging to 1 classes.
Found 41 images belonging to 1 classes.
Found 160 images belonging to 1 classes.
Found 41 images belonging to 1 classes.


In [26]:
train_generator =  pre_process.combine_generators(img_gen, mask_gen)
valid_generator = pre_process.combine_generators(valid_img_gen, valid_mask_gen)

In [27]:
# 1. Initialize the model 
input_shape = (928, 928, 3) # Height and Weight has to be multiples of 16
                            # Also three channels, because the CNN expects an 
                            # RGB image 
model = cnn.build_unet(input_shape)

# 2. See model architecture
model.compile(optimizer = Adam(lr = 1e-3), loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 928, 928, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 928, 928, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 928, 928, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [47]:
path = "model_for_nuclei.h5"
tensorboard = "logs"

history = cnn.fitting(model, train_generator, weightpath= path, tensorboard= tensorboard)

 4/20 [=====>........................] - ETA: 1:02:24 - loss: 0.6847 - accuracy: 0.6998

KeyboardInterrupt: 

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, "y", label="Training loss")
plt.plot(epochs, val_loss, "r", label="Validation loss")
plt.title("Training and validation loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
plt.plot(epochs, acc, "y", label="Training acc")
plt.plot(epochs, val_acc, "r", label="Validation acc")
plt.title("Training and validation accuracy")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [46]:
# 4. Count the number of nuclei of each image. 
nuclei_df = cnn.nuclei_segmentation("C:\\Users\\saraa\\TFM\\mask\\nuclei\\")
nuclei_df


,Number of Nuclei
Timepoint,
0,6
1,6
2,6
3,6
4,6
...,...
196,79
197,81
198,90
